In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
# LOAD ALL IMAGES:
dt = 5
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [3]:
idx = 1
# IMG1:
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
# IMG2:
img2=cv2.imread(dict[str(idx+2)])
craters_det2 = detect(img2)

In [4]:
# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)

In [5]:
def find_query(df_craters):
    STORED = []
    c=0
    M=df_craters.shape[0]
    QUERY = np.zeros((M**2,15))
    for i in range(M):
        for j in range(M):
            if i != j:
                triplet, HP, PICKS = find_triplet(df_craters, idx_a = i, idx_b=j)
                Names = [triplet[0].name,triplet[1].name,triplet[2].name]
                names = set(Names)
                if names not in STORED:
                    STORED.append(names)
                    K_v = compute_K_vet(triplet)
                    c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
                    c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
                    c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
                    QUERY[c] = np.hstack([K_v, Names,c1,c2,c3])
                    c+=1
                else: continue

        printProgressBar(i+1, df_craters.shape[0], printEnd='')
    QUERY = pd.DataFrame(QUERY, columns=['Angle1','Angle2','Angle3','ctr1','ctr2','ctr3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    return QUERY

In [16]:
QUERY1 = find_query(df_craters_det1)
QUERY2 = find_query(df_craters_det2)
# remove wrong triplet:
# index_names = QUERY1[ (QUERY1['Angle1'] == 0) | (QUERY1['Angle2'] == 0) | (QUERY1['Angle3'] == 0) \
index_names = QUERY1[(QUERY1['Angle1'] > 10) | (QUERY1['Angle2'] > 10) | (QUERY1['Angle3'] > 10) ].index
QUERY1.drop(index_names, inplace = True) 
index_names = QUERY2[ (QUERY2['Angle1'] == 0) | (QUERY2['Angle2'] == 0) | (QUERY2['Angle3'] == 0) \
             | (QUERY2['Angle1'] > 20) | (QUERY2['Angle2'] > 20) | (QUERY2['Angle3'] > 20)].index
QUERY2.drop(index_names, inplace = True) 

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [13]:
# Searching:
count,d, found = 0, 10, False
# for i in QUERY1.iloc:
for idx in range(QUERY1.shape[0]):
    if found:
        break

    i = QUERY1.iloc[idx]
    count+=1
    printProgressBar(count, QUERY1.shape[0], printEnd='')
    
    for idy in range(QUERY2.shape[0]):
        j = QUERY2.iloc[idy]
        a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
        b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
        x1,x2,x3 = i.r1,i.r2,i.r3
        y1,y2,y3 = j.r1,j.r2,j.r3
#       switch
        d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
        d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
        d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
#       switch
        r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
        r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
        r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

        d = min(d1,d2,d3)
        r = min(r1,r2,r3)

        if (d < 2) & (r < 2):
            print('\nFOUNDED')
            save = [idx, idy]
            print(save)
            found = True
            break
print('\n')
if found:
    a = QUERY1.iloc[save[0]]
    b = QUERY2.iloc[save[1]]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img1.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)

    plt.subplot(122)
    cp2 = img2.copy()
    IMG2 =  img_plus_crts(cp2, crts2, color="red")
    plt.imshow(IMG2)
    plt.show()
else:
    print("Match can't find any triplets.")



Match can't find any triplets.


In [14]:
QUERY1

,Angle1,Angle2,Angle3,ctr1,ctr2,ctr3,x1,y1,r1,x2,y2,r2,x3,y3,r3


In [8]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [9]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [10]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [11]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [12]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)